# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [2]:
gdf = cudf.read_csv('./data/week2.csv', usecols=['lat','long','infected'])

## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [3]:
hospitals = cudf.read_csv('./data/hospitals.csv', usecols=['Latitude','Longitude'])
clinics = cudf.read_csv('./data/clinics.csv', usecols=['Latitude','Longitude'])
all_med = cudf.concat([hospitals, clinics])

In [4]:
print(hospitals.shape)
print(clinics.shape)

(1229, 2)
(19082, 2)


In [5]:
all_med

,Latitude,Longitude
0,51.379997,-0.406042
1,51.315132,-0.556289
2,51.437195,-2.847193
3,53.459743,-2.245469
4,52.078121,-0.030604
...,...,...
19077,53.737923,-1.767982
19078,51.341579,-2.784808
19079,51.286285,-2.820388
19080,53.163048,-2.205781


Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [6]:
all_med.dropna(axis=0, inplace=True)

## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [7]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [9]:
all_med['northing'], all_med['easting'] = latlong2osgbgrid_cupy(cp.asarray(all_med['Latitude']), cp.asarray(all_med['Longitude']))

## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [12]:
knn = cuml.NearestNeighbors(n_neighbors=1)
knn.fit(all_med[['northing','easting']])

NearestNeighbors(n_neighbors=1, verbose=4, handle=<cuml.common.handle.Handle object at 0x7faa4d267a10>, algorithm='brute', metric='euclidean', p=2, metric_params=None, output_type='cudf')

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [27]:
infected_gdf = gdf[gdf['infected'] == 1.0].reset_index(drop=True)

Create `northing` and `easting` values for `infected_gdf`.

In [28]:
infected_df['northing'], infected_df['easting'] = latlong2osgbgrid_cupy(cp.asarray(infected_df['lat']), cp.asarray(infected_df['long']))

Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [29]:
distance, indices = knn.kneighbors(infected_df[['northing','easting']], n_neighbors=1)

In [30]:
distance

0         2566.392090
1         2990.930420
2         1144.866821
3         1173.139404
4         1470.608032
             ...     
70875     8617.926758
70876     1311.609863
70877     7633.995605
70878     7071.348145
70879    10223.186523
Length: 70880, dtype: float32

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [31]:
indices.head()

0    18316
1    12816
2     4489
3     4489
4     4962
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [32]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat         53.715826
long        -2.430079
infected     1.000000
Name: 0, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [ ]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

Please proceed to [the next notebook](3-04_identify_risk_factors.ipynb).